# Beat Analyzer

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wave, sys
   
raw = wave.open("worlds_smallest_violin.wav")

signal = raw.readframes(-1)
signal = np.frombuffer(signal, dtype ="int16")

f_rate = raw.getframerate()

time = np.linspace(0, len(signal)/f_rate, num = len(signal))

plt.title("Sound Wave")
plt.xlabel("Time")
plt.plot(time[:int(5e5)], signal[:int(5e5)])
plt.show()

## Spectogram

Using matplotlib's [`specgram`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.specgram.html#matplotlib.pyplot.specgram) function, we can find the frequencies within times. 

In [ ]:
ranges = np.array([0, -1], dtype=int)
print(f"Time range = {time[ranges[0]]} - {time[ranges[1]]}")

result = plt.specgram(signal[ranges[0]:ranges[1]], NFFT=256, Fs=f_rate, cmap="jet")
plt.colorbar()
plt.show()

In [ ]:
spectrum = result[0]
dt = result[2][1] - result[2][0]

magnitudes = np.sum(spectrum, axis=0) # axis = 0, column addition

plt.plot(result[2][1:-1] + time[ranges[0]], magnitudes[1:-1])

plt.xlabel("Time (s)")
plt.ylabel("Magnitude")
plt.show()

In [ ]:
from scipy.signal import find_peaks

# https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-python-numpy-scipy
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

magnitudes_rolling = moving_average(magnitudes, 30)
magnitudes_rolling_norm = magnitudes_rolling/np.max(magnitudes_rolling)

peaks = find_peaks(magnitudes_rolling_norm, 
                   distance=0.3/(dt)
                  )

plt.plot((result[2] + time[ranges[0]])[:len(magnitudes_rolling_norm)], magnitudes_rolling_norm)
plt.scatter(result[2][peaks[0]] + time[ranges[0]], magnitudes_rolling_norm[peaks[0]], color="tab:orange")
plt.show()

In [ ]:
result[2][2] - result[2][1]

In [ ]:
peak_times = result[2][peaks[0]] + time[ranges[0]]
print("Peak times in seconds", peak_times.tolist())

## References

1. [Tempo and Beat Tracking, www.youtube.com](https://www.youtube.com/watch?v=FmwpkdcAXl0&t=576s)